In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pydicom

In [33]:
def calculate_bpe(pre_contrast_path, post_contrast_path):
    
    pre_contrast_dicom = pydicom.dcmread(pre_contrast_path)

    post_contrast_dicom = pydicom.dcmread(post_contrast_path)

    pre_contrast_image = pre_contrast_dicom.pixel_array.astype(np.float32)

    post_contrast_image = post_contrast_dicom.pixel_array.astype(np.float32) 

    pre_contrast_image[pre_contrast_image == 0] = np.finfo(np.float32).eps
    
    bpe = ((post_contrast_image-pre_contrast_image ) / pre_contrast_image) * 100


    return bpe

In [25]:
pre_path = 'data/manifest-1722210581008/ISPY2/ISPY2-100899/10-26-2002-100899T0-ISPY2MRIT0-88595/500.000000-ISPY2 AX VIBRANT-SERIES-52523/1-042.dcm'
post_path = 'data/manifest-1722210581008/ISPY2/ISPY2-100899/10-26-2002-100899T0-ISPY2MRIT0-88595/501.000000-ISPY2 Ph1AX VIBRANT-SERIES-03969/1-042.dcm'


In [34]:

bpe_image = calculate_bpe(pre_path, post_path)

# Calculate mean BPE percentage
mean_bpe = np.mean(bpe_image)
print(f"BPE Percentage: {mean_bpe}%")

BPE Percentage: 5961607.0%
